#### Import libraries

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib
from scipy import ndimage
from deepbrain import Extractor
from tensorflow import keras
from tensorflow.keras import layers
import time
import cv2
import shutil

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme(context='notebook')
sns.set_style("ticks")

#### Functions

In [2]:
def read_nifti_file(file):
    """
    Read and load nifti file.
    """
    
    # Read file
    volume = nib.load(file)
    
    # Get raw data
    volume = volume.get_fdata()
    
    # Exchange axis 0 and 2
    if volume.shape[1] == volume.shape[2]:
        print(f"{file} has a shape incompatible")
    
    return volume


def remove_skull(volume):
    """
    Extract only brain mass from volume.
    """
    
    # Initialize brain tissue extractor
    ext = Extractor()

    # Calculate probability of being brain tissue
    prob = ext.run(volume) 

    # Extract mask with probability higher than 0.5
    mask = prob > 0.5
    
    # Detect only pixels with brain mass
    volume [mask == False] = 0
    volume = volume.astype("float32")
    
    return volume


def normalize(volume):
    """
    Normalize the volume intensity.
    """
    
    I_min = np.amin(volume)
    I_max = np.amax(volume)
    new_min = 0.0
    new_max = 1.0
    
    volume_nor = (volume - I_min) * (new_max - new_min)/(I_max - I_min)  + new_min
    volume_nor = volume_nor.astype("float32")
    
    return volume_nor


def cut_volume(volume):
    """
    Cut size of 3D volume.
    """
    
    if volume.shape[0] == 256:
        volume_new = volume[20:220,30:,:]
    
    if volume.shape[0] == 192:
        volume_new = volume[20:180,20:180,:]
    
    return volume_new


def resize_volume(volume):
    """
    Resize across z-axis
    """
    
    # Set the desired depth
    desired_height = 180
    desired_width = 180
    desired_depth = 110
    
    # Get current depth
    current_height = volume.shape[0]
    current_width = volume.shape[1]
    current_depth = volume.shape[2]
    
    # Compute depth factor
    height = current_height / desired_height
    width = current_width / desired_width
    depth = current_depth / desired_depth

    height_factor = 1 / height
    width_factor = 1 / width
    depth_factor = 1 / depth
    
    # Rotate
    #img = ndimage.rotate(img, 90, reshape=False)
    
    # Resize across z-axis
    volume = ndimage.zoom(volume, (height_factor, width_factor, depth_factor), order=1)
    
    return volume


def save_matrix(volume, file, save_path = "./Volume_files/"):
    """
    Save 3D matrix into numpy file.
    """
    
    # Check if save_path folder exists
    if not os.path.exists(save_path):
        os.mkdir(save_path)   

    # Get Image ID from path
    title = file.split("_")[-1].split(".")[0]
    
    # Save volume array
    if os.path.exists(save_path + title + ".npz"):
        print(f"    {title} was already processed.")
    else:
        np.savez_compressed(save_path + title, volume)
    

def process_scan(file, save_path):
    """
    Read, skull stripping and resize Nifti file.
    """
    
    # Read Nifti file
    volume = read_nifti_file(file)
    
    # Extract skull from 3D volume
    volume = remove_skull(volume)
    
    # Cut 3D volume
    #volume = cut_volume(volume)
    
    # Resize width, height and depth
    volume = resize_volume(volume)
    
    # Normalize pixel intensity
    volume = normalize(volume)
    
    # Save 3D matrix
    save_matrix(volume, file, save_path)
    
    return volume

#### Specify paths

In [3]:
list_path_mri = ["../Datasets/Files_network/"]
save_path = "../Datasets/Volume_files/"

#### Load list of Images IDs

In [4]:
list_titles_path = '../Datasets/list_titles_cleaned.npz'

In [5]:
list_titles = np.load(list_titles_path, allow_pickle= True)
list_titles = list_titles['arr_0']
list_titles = list(list_titles)

#### Main

In [6]:
start_time = time.time()

count_volumes = 0

for path in list_path_mri:
    
    print("[+] Current path:", path)
    
    for file in os.listdir(path):
        
        # Avoid trigerring .DS_Store (when use macOS)
        if file.startswith('.DS_Store'):
            continue
        
        title = file.split("_")[-1].split(".")[0]

        if title in list_titles:
            
            # Volume process
            volume = process_scan(path + file, save_path)

            count_volumes += 1

            # Print counter status
            if(count_volumes % 50 == 0):
                print(f"    {count_volumes} volumes processed")
        
print("[+] Total number of volumes processed:", count_volumes)

end_time = time.time()
print("\n[+] Time of process: "+"{:.2f}".format(end_time-start_time));

[+] Current path: ../Datasets/Files_network/
Instructions for updating:
Use tf.gfile.GFile.
    50 volumes processed
    100 volumes processed
    150 volumes processed
    200 volumes processed
    250 volumes processed
    300 volumes processed
    350 volumes processed
    400 volumes processed
    450 volumes processed
    500 volumes processed
    550 volumes processed
    600 volumes processed
    650 volumes processed
    700 volumes processed
    750 volumes processed
    800 volumes processed
    850 volumes processed
    900 volumes processed
    950 volumes processed
    1000 volumes processed
    1050 volumes processed
[+] Total number of volumes processed: 1085

[+] Time of process: 7981.61


#### Others

In [ ]:
counter = 0
shapes = []
counters = [0] * 30
titles_shapes = [[] for i in range(30)]

for index, file in enumerate(os.listdir(new_path)):
    
    # Avoid trigerring .DS_Store (when use macOS)
    if subpath.startswith('.DS_Store'):
        continue
    
    # Read Nifti file
    volume = read_nifti_file(new_path + file)
    
    if volume.shape not in shapes:
        shapes.append(volume.shape)
    
    index_shapes = shapes.index(volume.shape)
    
    counters[index_shapes] += 1 
    
    titles_shapes[index_shapes].append(file.split("_")[-1].split(".")[0])
    
    if index % 50 == 0:
        print(index)

In [ ]:
list_shapes_cleaned = [shape_ for index, shape_ in enumerate(shapes) if counters[index] > 30]
list_counters_cleaned = [counter_ for index, counter_ in enumerate(counters) if counters[index] > 30]
list_titles_cleaned = [title_ for index, title_ in enumerate(titles_shapes) if counters[index] > 30]
list_titles_cleaned = [item for sublist in list_titles_cleaned for item in sublist]

711 from 1100 has shape (256,256) = 64%

In [ ]:
list_shapes_cleaned

In [ ]:
list_counters_cleaned

#### Others

In [ ]:
# Show initial image before being resized
image_ = volume[:,:,40]
print("[+] Shape of the image:", image_.shape)
plt.figure(figsize = (5, 4))
plt.imshow(image_, cmap = "gray");

In [ ]:
height = 180
width = 180
depth = volume.shape[2]

volume_resized = np.zeros((height, width, depth))

for index in range(depth):
    
    # Get 2D slice
    image = volume[:, :, index]
    
    # Resize 2D slice
    image_resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)
    
    volume_resized[:, :, index] = image_resized

In [ ]:
height = 180
width = 180
depth = 100

volume_resized_2 = np.zeros((height, width, depth))

for index in range(width):
    
    # Get 2D slice
    image = volume_resized[:, index, :]
    
    # Resize 2D slice
    image_resized = cv2.resize(image, (depth, height), interpolation=cv2.INTER_CUBIC)
    
    volume_resized_2[:, index, :] = image_resized

In [ ]:
# Load volume array
volume_loaded_2 = np.load(save_path + "MR.npz" , allow_pickle= True)
volume_loaded_2 = volume_loaded_2['arr_0']

In [ ]:
# Save volume as nifti file
nifti_image = nib.Nifti1Image(volume_loaded , affine=np.eye(4))
nib.save(nifti_image, "../Datasets/nifti_created_10.nii.gz") 

In [ ]:
# https://www.thepythoncode.com/article/contour-detection-opencv-python
# Get 2D slice
image_slice = volume[:,:,60]
image_slice = np.float32(image_slice)
data = im.fromarray(image_slice)

# Create a binary thresholded image
_, binary = cv2.threshold(np.float32(data), 0.5, 1, cv2.THRESH_BINARY_INV)

# Find contours
contours, hierarchy = cv2.findContours(image = np.array(binary, dtype=np.int32),  
                                       mode = cv2.RETR_FLOODFILL, 
                                       method = cv2.CHAIN_APPROX_SIMPLE)

# Draw contours
cnt = contours[4]
image_contours = cv.drawContours(np.float32(data), [cnt], 0, (0,255,0), 3)
plt.imshow(image_contours, c)
plt.show()